In [1]:
import os
os.chdir("../")

In [74]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [75]:
df = pd.read_csv("Artificats/Time_Estimation_Dataset/train.csv")

In [76]:
df.head()

,ID,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken
0,0x4607,37.0,4.9,22.765049,75.912471,2022-03-19,11:30:00,Sunny,High,2,motorcycle,0.0,Urban,24
1,0xb379,34.0,4.5,13.043041,77.813237,2022-03-25,19:45:00,Stormy,Jam,2,scooter,1.0,Metropolitian,33
2,0x5d6d,23.0,4.4,12.924264,77.688400,2022-03-19,08:30:00,Sandstorms,Low,0,motorcycle,1.0,Urban,26
3,0x7a6a,38.0,4.7,11.053669,77.026494,2022-04-05,18:00:00,Sunny,Medium,0,motorcycle,1.0,Metropolitian,21
4,0x70a2,32.0,4.6,13.012793,80.289982,2022-03-26,13:30:00,Cloudy,High,1,scooter,1.0,Metropolitian,30


In [77]:
df.drop(['ID'], axis=1, inplace=True)

In [78]:
df['Order_Date']=pd.to_datetime(df['Order_Date'])

# Creating three column for day,month and year
df['Order_day']=df['Order_Date'].dt.day
df['Order_month']=df['Order_Date'].dt.month
df['Order_year']=df['Order_Date'].dt.year

In [79]:
df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'])

# Creating two new column for hour and minute
df['Hour_order']=df['Time_Orderd'].dt.hour
df['Min_order']=df['Time_Orderd'].dt.minute

C:\Users\rahul\AppData\Local\Temp\ipykernel_2120\1731031211.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Time_Orderd'] = pd.to_datetime(df['Time_Orderd'])


In [80]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Time_taken,Order_day,Order_month,Order_year,Hour_order,Min_order
0,37.0,4.9,22.765049,75.912471,2022-03-19,2025-01-17 11:30:00,Sunny,High,2,motorcycle,0.0,Urban,24,19,3,2022,11.0,30.0
1,34.0,4.5,13.043041,77.813237,2022-03-25,2025-01-17 19:45:00,Stormy,Jam,2,scooter,1.0,Metropolitian,33,25,3,2022,19.0,45.0
2,23.0,4.4,12.924264,77.688400,2022-03-19,2025-01-17 08:30:00,Sandstorms,Low,0,motorcycle,1.0,Urban,26,19,3,2022,8.0,30.0
3,38.0,4.7,11.053669,77.026494,2022-04-05,2025-01-17 18:00:00,Sunny,Medium,0,motorcycle,1.0,Metropolitian,21,5,4,2022,18.0,0.0
4,32.0,4.6,13.012793,80.289982,2022-03-26,2025-01-17 13:30:00,Cloudy,High,1,scooter,1.0,Metropolitian,30,26,3,2022,13.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,2022-03-24,2025-01-17 11:35:00,Windy,High,1,motorcycle,0.0,Metropolitian,32,24,3,2022,11.0,35.0
45589,21.0,4.6,0.070000,0.070000,2022-02-16,2025-01-17 19:55:00,Windy,Jam,0,motorcycle,1.0,Metropolitian,36,16,2,2022,19.0,55.0
45590,30.0,4.9,13.052394,80.272439,2022-03-11,2025-01-17 23:50:00,Cloudy,Low,1,scooter,0.0,Metropolitian,16,11,3,2022,23.0,50.0
45591,20.0,4.7,11.041753,77.026241,2022-03-07,2025-01-17 13:35:00,Cloudy,High,0,motorcycle,1.0,Metropolitian,26,7,3,2022,13.0,35.0


In [81]:
df.drop(["Time_Orderd", "Order_Date"],axis = 1, inplace= True)

In [82]:
df.shape

(45593, 16)

In [83]:
column = df.pop("Time_taken")
df.insert(15, column.name, column)

In [84]:
def create_delivery_features(df):

    df_features = df
    
    df_features['avg_delivery_time_area'] = df_features.groupby('City')['Time_taken'].transform('mean')

    df_features['traffic_weather_impact'] = df_features.groupby(
        ['Road_traffic_density', 'Weatherconditions']
    )['Time_taken'].transform('mean')

    df_features['vehicle_capacity_utilization'] = (
        df_features['multiple_deliveries'] / 
        df_features.groupby('Type_of_vehicle')['multiple_deliveries'].transform('max')
    ).fillna(0)
    
    return df_features
    
df = create_delivery_features(df)

In [85]:
col = df.pop("Time_taken")
df.insert(18, col.name, col)

In [86]:
df1 = df.copy()

In [87]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,City,Order_day,Order_month,Order_year,Hour_order,Min_order,avg_delivery_time_area,traffic_weather_impact,vehicle_capacity_utilization,Time_taken
0,37.0,4.9,22.765049,75.912471,Sunny,High,2,motorcycle,0.0,Urban,19,3,2022,11.0,30.0,22.984017,23.448980,0.000000,24
1,34.0,4.5,13.043041,77.813237,Stormy,Jam,2,scooter,1.0,Metropolitian,25,3,2022,19.0,45.0,27.315226,29.850194,0.333333,33
2,23.0,4.4,12.924264,77.688400,Sandstorms,Low,0,motorcycle,1.0,Urban,19,3,2022,8.0,30.0,22.984017,20.297049,0.333333,26
3,38.0,4.7,11.053669,77.026494,Sunny,Medium,0,motorcycle,1.0,Metropolitian,5,4,2022,18.0,0.0,27.315226,20.195518,0.333333,21
4,32.0,4.6,13.012793,80.289982,Cloudy,High,1,scooter,1.0,Metropolitian,26,3,2022,13.0,30.0,27.315226,28.940860,0.333333,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,Windy,High,1,motorcycle,0.0,Metropolitian,24,3,2022,11.0,35.0,27.315226,26.972789,0.000000,32
45589,21.0,4.6,0.070000,0.070000,Windy,Jam,0,motorcycle,1.0,Metropolitian,16,2,2022,19.0,55.0,27.315226,30.219056,0.333333,36
45590,30.0,4.9,13.052394,80.272439,Cloudy,Low,1,scooter,0.0,Metropolitian,11,3,2022,23.0,50.0,27.315226,22.208445,0.000000,16
45591,20.0,4.7,11.041753,77.026241,Cloudy,High,0,motorcycle,1.0,Metropolitian,7,3,2022,13.0,35.0,27.315226,28.940860,0.333333,26


In [88]:
df.isna().sum()

Delivery_person_Age             1854
Delivery_person_Ratings         1908
Delivery_location_latitude         0
Delivery_location_longitude        0
Weatherconditions                616
Road_traffic_density             601
Vehicle_condition                  0
Type_of_vehicle                    0
multiple_deliveries              993
City                            1200
Order_day                          0
Order_month                        0
Order_year                         0
Hour_order                      1731
Min_order                       1731
avg_delivery_time_area          1200
traffic_weather_impact           616
vehicle_capacity_utilization       0
Time_taken                         0
dtype: int64

In [89]:
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [90]:
mean_features = ["Delivery_person_Age","Delivery_person_Ratings","avg_delivery_time_area"]
mode_features = ["Weatherconditions","multiple_deliveries","traffic_weather_impact",
    "Road_traffic_density","City","Order_day","Order_month","Order_year","Hour_order","Min_order",]
OE_features = ["Vehicle_condition",'Weatherconditions', 'Type_of_vehicle',"Road_traffic_density"]
OHE_features = ["City"]


In [91]:
SI_mean = SimpleImputer(strategy="mean")
SI_mode = SimpleImputer(strategy="most_frequent")
OE = OrdinalEncoder()
OHE = OneHotEncoder()

preprocessor = ColumnTransformer(
    (
        ("mean_features", SI_mean, mean_features),
        ("mode_features", SI_mode, mode_features),
        ("OE_features", OE, OE_features),
        ("OHE_features", OHE, OHE_features)
    )
)

In [92]:
# 1. Handle mean and mode imputation
df[mean_features] = SI_mean.fit_transform(df[mean_features])
df[mode_features] = SI_mode.fit_transform(df[mode_features])

# 2. Handle ordinal encoding
df[OE_features] = OE.fit_transform(df[OE_features])

# 3. Handle one-hot encoding separately
ohe_transformed = OHE.fit_transform(df[OHE_features])
ohe_feature_names = OHE.get_feature_names_out(OHE_features)

# 4. Convert sparse matrix to DataFrame
ohe_df = pd.DataFrame(
    ohe_transformed.toarray(),
    columns=ohe_feature_names,
    index=df.index
)

# 5. Drop original OHE columns and join encoded ones
df = df.drop(columns=OHE_features)
df = pd.concat([df, ohe_df], axis=1)

In [93]:
column = df.pop("Time_taken")
df.insert(20, column.name, column)

In [94]:
df

,Delivery_person_Age,Delivery_person_Ratings,Delivery_location_latitude,Delivery_location_longitude,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_vehicle,multiple_deliveries,Order_day,...,Order_year,Hour_order,Min_order,avg_delivery_time_area,traffic_weather_impact,vehicle_capacity_utilization,City_Metropolitian,City_Semi-Urban,City_Urban,Time_taken
0,37.0,4.9,22.765049,75.912471,4.0,0.0,2.0,2.0,0.0,19,...,2022,11.0,30.0,22.984017,23.44898,0.000000,0.0,0.0,1.0,24
1,34.0,4.5,13.043041,77.813237,3.0,1.0,2.0,3.0,1.0,25,...,2022,19.0,45.0,27.315226,29.850194,0.333333,1.0,0.0,0.0,33
2,23.0,4.4,12.924264,77.688400,2.0,2.0,0.0,2.0,1.0,19,...,2022,8.0,30.0,22.984017,20.297049,0.333333,0.0,0.0,1.0,26
3,38.0,4.7,11.053669,77.026494,4.0,3.0,0.0,2.0,1.0,5,...,2022,18.0,0.0,27.315226,20.195518,0.333333,1.0,0.0,0.0,21
4,32.0,4.6,13.012793,80.289982,0.0,0.0,1.0,3.0,1.0,26,...,2022,13.0,30.0,27.315226,28.94086,0.333333,1.0,0.0,0.0,30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45588,30.0,4.8,26.912328,75.804257,5.0,0.0,1.0,2.0,0.0,24,...,2022,11.0,35.0,27.315226,26.972789,0.000000,1.0,0.0,0.0,32
45589,21.0,4.6,0.070000,0.070000,5.0,1.0,0.0,2.0,1.0,16,...,2022,19.0,55.0,27.315226,30.219056,0.333333,1.0,0.0,0.0,36
45590,30.0,4.9,13.052394,80.272439,0.0,2.0,1.0,3.0,0.0,11,...,2022,23.0,50.0,27.315226,22.208445,0.000000,1.0,0.0,0.0,16
45591,20.0,4.7,11.041753,77.026241,0.0,0.0,0.0,2.0,1.0,7,...,2022,13.0,35.0,27.315226,28.94086,0.333333,1.0,0.0,0.0,26


In [95]:
df.to_csv("testing.cvs")

In [96]:
X = df.drop("Time_taken", axis=1)
y = df["Time_taken"]

In [98]:
from sklearn.preprocessing import StandardScaler

ss = StandardScaler()

X_transformed = ss.fit_transform(X)
y_transformed = ss.fit_transform(y.values.reshape(-1, 1))

In [102]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

X_train, X_test, y_train, y_test = train_test_split(X_transformed, y_transformed, test_size=0.2, random_state=42)


In [103]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred_lr = lr.predict(X_test)

# mean_squared_error(y_test, y_pred_lr)
r2_score(y_test, y_pred_lr)

0.5600575010959574

In [104]:
rf = RandomForestRegressor()
rf.fit(X_train, y_train)

y_pred_rf = rf.predict(X_test)

# mean_squared_error(y_test, y_pred_rf)
r2_score(y_test, y_pred_rf)

r:\-real-time-route-optimization-and-predictive-analytics\.venv\Lib\site-packages\sklearn\base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


0.7438494343222923